In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj, transform, CRS
from shapely.geometry import Polygon, Point

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# display full
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import geopandas as gpd

In [3]:
import geoplot as gplt

In [56]:
files = {
    'tracts' : '../resource_files/bexar_county/Bexar_County_Census_Tracts-shp/Bexar_County_Census_Tracts.shp', 
    'block_groups' : '../resource_files/bexar_county/Bexar_County_Census_Block_Groups-shp/Bexar_County_Census_Block_Groups.shp', 
    'blocks' : '../resource_files/bexar_county/Bexar_County_Census_Blocks-shp/Bexar_County_Census_Blocks.shp', 
    'stops_201909' :  '../resource_files/via_201909/stops.txt', 
    'stops_202004' :  '../resource_files/via_202004/stops.csv', 
}

In [5]:
gdf_tracts = gpd.read_file(files['tracts']).set_index('TRACT')
gdf_tracts['density']=gdf_tracts.SUM_POPULA / gdf_tracts.ShapeSTAre.min()
gdf_tracts = gdf_tracts.to_crs('epsg:4326')
gdf_tracts.head()

,OBJECTID,SUM_POPULA,ShapeSTAre,ShapeSTLen,geometry,density
TRACT,,,,,,
48029110100,1,3379,3.816858e+07,26074.749275,"POLYGON ((-98.48695 29.43502, -98.48626 29.434...",0.000433
48029110300,2,2542,1.667905e+07,22363.990201,"POLYGON ((-98.47326 29.41461, -98.47326 29.413...",0.000325
48029110500,3,2238,1.366653e+07,16048.493069,"POLYGON ((-98.50799 29.42314, -98.50812 29.422...",0.000287
48029110600,4,7553,2.117379e+07,20426.412628,"POLYGON ((-98.50111 29.42735, -98.50121 29.426...",0.000967
48029110700,5,1398,1.058371e+07,16998.776148,"POLYGON ((-98.50401 29.44202, -98.50386 29.441...",0.000179


In [6]:
gdf_block_groups = gpd.read_file(files['block_groups']).set_index('BLOCKGROUP')
gdf_block_groups['density']=gdf_block_groups.SUM_POPULA / gdf_block_groups.ShapeSTAre
gdf_block_groups = gdf_block_groups.to_crs('epsg:4326')
gdf_block_groups.head()

,OBJECTID,SUM_POPULA,Shape_STAr,Shape_STLe,ShapeSTAre,ShapeSTLen,geometry,density
BLOCKGROUP,,,,,,,,
480291101001,1,955,1.951706e+07,23279.128568,1.951706e+07,23279.128568,"POLYGON ((-98.48695 29.43502, -98.48626 29.434...",0.000049
480291101002,2,694,7.319464e+06,14609.297870,7.319464e+06,14609.297870,"POLYGON ((-98.48824 29.41933, -98.48826 29.418...",0.000095
480291101003,3,1730,1.133205e+07,14546.232289,1.133205e+07,14546.232289,"POLYGON ((-98.49432 29.43370, -98.49426 29.433...",0.000153
480291103001,4,1041,4.292228e+06,9662.074836,4.292228e+06,9662.074836,"POLYGON ((-98.47984 29.40970, -98.47983 29.409...",0.000243
480291103002,5,797,4.074139e+06,10523.204970,4.074139e+06,10523.204970,"POLYGON ((-98.47989 29.40860, -98.47992 29.408...",0.000196


In [7]:
gdf_blocks_cols = [
    'OBJECTID', 'ID', 'COLORING', 'BLOCKGROUP', 'TRACT', 'MCD', 
    'PLACE', 'VTD', 'CONGRESS', 'LOWERSLD', 'UPPERSLD', 'UNIFSCHOOL',
    'POPULATION', 'HISPANIC_O', 'NH_WHT', 'NH_BLK', 'NH_ASN', 'NH_OTH',
    'gecovector', 'ShapeSTAre', 'ShapeSTLen', 'geometry'
]

In [8]:
gdf_blocks = gpd.read_file(files['blocks']).set_index('BLOCK')
gdf_blocks['NH_OTH'] = (
    gdf_blocks.POPULATION 
    - gdf_blocks.HISPANIC_O 
    - gdf_blocks.NH_WHT 
    - gdf_blocks.NH_BLK 
    - gdf_blocks.NH_ASN
)
# gdf_blocks['density']=gdf_blocks.POPULATION / gdf_blocks.ShapeSTAre
gdf_blocks = gdf_blocks[gdf_blocks_cols]
gdf_blocks = gdf_blocks.to_crs('epsg:4326')
gdf_blocks.head()

,OBJECTID,ID,COLORING,BLOCKGROUP,TRACT,MCD,PLACE,VTD,CONGRESS,LOWERSLD,UPPERSLD,UNIFSCHOOL,POPULATION,HISPANIC_O,NH_WHT,NH_BLK,NH_ASN,NH_OTH,gecovector,ShapeSTAre,ShapeSTLen,geometry
BLOCK,,,,,,,,,,,,,,,,,,,,,,
480291919003005,1,1400310,4,480291919003,48029191900,4802993407,4865000,480294004,4821,48120,48019,4838730,13,11,0,0,0,2,0.001038,28896.121094,1063.271402,"POLYGON ((-98.46781 29.42506, -98.46783 29.424..."
480291411011002,2,1400328,3,480291411011,48029141101,4802993407,4865000,480291074,4823,48119,48019,4838730,0,0,0,0,0,0,0.018304,509512.736328,8615.618889,"POLYGON ((-98.46881 29.36511, -98.46815 29.364..."
480291411011006,3,1400348,3,480291411011,48029141101,4802993407,4865000,480291074,4823,48119,48019,4838730,102,82,16,2,0,2,0.012803,356394.378906,2836.366689,"POLYGON ((-98.46323 29.36640, -98.46324 29.365..."
480291410002007,4,1400365,1,480291410002,48029141000,4802993407,4865000,480291074,4823,48119,48019,4838730,162,149,13,0,0,0,0.012892,358877.382812,2852.304380,"POLYGON ((-98.46667 29.36662, -98.46668 29.367..."
480291410002006,5,1400383,3,480291410002,48029141000,4802993407,4865000,480291074,4823,48119,48019,4838730,133,124,5,4,0,0,0.016129,448943.710938,2994.127235,"POLYGON ((-98.46668 29.36753, -98.46667 29.368..."


In [22]:
# For simplification, using distance per degree latitude to determine buffer amount. 
# This is slightly inaccurate due to the curvature of the earth. Buffer distances is
# set at 800 meters, which is roughly a half mile.

meters_per_degree = 111111
buffer_in_meters = 750
buffer_in_degrees = buffer_in_meters / meters_per_degree
buffer_in_degrees

0.00675000675000675

In [52]:
def wrangle_stops(source_file, buffer=.01):
    
    keep_cols = [
        'stop_code', 'stop_name', 'stop_lat', 'stop_lon', 'wheelchair_boarding'
    ]
    
    gdf =pd.read_csv(source_file).set_index('stop_id')
    gdf = gdf[keep_cols]
    gdf = gpd.GeoDataFrame(
        gdf,
        geometry = gpd.points_from_xy(gdf.stop_lon, gdf.stop_lat),
        crs={'epsg:4326'}
    )
    
    gdf_buff = gdf[['geometry']].copy()
    
    gdf_buff['geometry'] = gdf_buff.buffer(buffer)
    gdf_buff = gdf_buff.rename(columns={'geometry':'geo_poly'})
    
    gdf = gdf.rename(columns={'geometry':'geo_point'})
    
    gdf = gdf.join(gdf_buff, how='left')
    gdf = gdf.set_geometry('geo_poly')
    
    return gdf


In [54]:
gdf_stops_202004 = wrangle_stops(files['stops_202004'], buffer=buffer_in_degrees)

gdf_stops_202004.head()

,stop_code,stop_name,stop_lat,stop_lon,wheelchair_boarding,geo_point,geo_poly
stop_id,,,,,,,
25316,25316,BLANCO & DRESDEN,29.499014,-98.507783,0,POINT (-98.50778 29.49901),"POLYGON ((-98.50103 29.49901, -98.50107 29.498..."
72479,72479,FRESNO & IH-10 W ACCESS RD.,29.474488,-98.516238,0,POINT (-98.51624 29.47449),"POLYGON ((-98.50949 29.47449, -98.50952 29.473..."
56239,56239,FREDERICKSBURG RD. & N. FLORES,29.441883,-98.503773,0,POINT (-98.50377 29.44188),"POLYGON ((-98.49702 29.44188, -98.49706 29.441..."
76759,76759,MARTIN & N. FRIO,29.430081,-98.503058,0,POINT (-98.50306 29.43008),"POLYGON ((-98.49631 29.43008, -98.49634 29.429..."
88973,88973,DOLOROSA & S. FLORES,29.424073,-98.494741,0,POINT (-98.49474 29.42407),"POLYGON ((-98.48799 29.42407, -98.48802 29.423..."


In [64]:
gdf_stops_202004.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6491 entries, 25316 to 74369
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   stop_code            6491 non-null   int64   
 1   stop_name            6491 non-null   object  
 2   stop_lat             6491 non-null   float64 
 3   stop_lon             6491 non-null   float64 
 4   wheelchair_boarding  6491 non-null   int64   
 5   geo_point            6491 non-null   geometry
 6   geo_poly             6491 non-null   geometry
dtypes: float64(2), geometry(2), int64(2), object(1)
memory usage: 405.7+ KB


In [57]:
gdf_stops_201909 = wrangle_stops(files['stops_201909'], buffer=buffer_in_degrees)

gdf_stops_201909.head()

,stop_code,stop_name,stop_lat,stop_lon,wheelchair_boarding,geo_point,geo_poly
stop_id,,,,,,,
25316,25316,BLANCO & DRESDEN,29.499014,-98.507783,0,POINT (-98.50778 29.49901),"POLYGON ((-98.50103 29.49901, -98.50107 29.498..."
72479,72479,FRESNO & IH-10 W ACCESS RD.,29.474488,-98.516238,0,POINT (-98.51624 29.47449),"POLYGON ((-98.50949 29.47449, -98.50952 29.473..."
56239,56239,FREDERICKSBURG RD. & N. FLORES,29.441883,-98.503773,0,POINT (-98.50377 29.44188),"POLYGON ((-98.49702 29.44188, -98.49706 29.441..."
76759,76759,MARTIN & N. FRIO,29.430081,-98.503058,0,POINT (-98.50306 29.43008),"POLYGON ((-98.49631 29.43008, -98.49634 29.429..."
88973,88973,DOLOROSA & S. FLORES,29.424073,-98.494741,0,POINT (-98.49474 29.42407),"POLYGON ((-98.48799 29.42407, -98.48802 29.423..."


In [65]:
gdf_stops_201909.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6895 entries, 25316 to 56683
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   stop_code            6895 non-null   int64   
 1   stop_name            6895 non-null   object  
 2   stop_lat             6895 non-null   float64 
 3   stop_lon             6895 non-null   float64 
 4   wheelchair_boarding  6895 non-null   int64   
 5   geo_point            6895 non-null   geometry
 6   geo_poly             6895 non-null   geometry
dtypes: float64(2), geometry(2), int64(2), object(1)
memory usage: 430.9+ KB


In [63]:
gdf_stops_202004_blocks = gpd.sjoin(gdf_blocks, gdf_stops_202004, how='inner', op='intersects')
gdf_stops_202004_blocks = gdf_stops_202004_blocks.rename(columns={'index_right':'stop_id'})
gdf_stops_202004_blocks.head()

,OBJECTID,ID,COLORING,BLOCKGROUP,TRACT,MCD,PLACE,VTD,CONGRESS,LOWERSLD,UPPERSLD,UNIFSCHOOL,POPULATION,HISPANIC_O,NH_WHT,NH_BLK,NH_ASN,NH_OTH,gecovector,ShapeSTAre,ShapeSTLen,geometry,stop_id,stop_code,stop_name,stop_lat,stop_lon,wheelchair_boarding,geo_point
480291919003005,1,1400310,4,480291919003,48029191900,4802993407,4865000,480294004,4821,48120,48019,4838730,13,11,0,0,0,2,0.001038,28896.121094,1063.271402,"POLYGON ((-98.46781 29.42506, -98.46783 29.424...",95783,95783,E. COMMERCE & S. HACKBERRY,29.42051,-98.473362,0,POINT (-98.47336 29.42051)
480291919001059,15233,1398520,3,480291919001,48029191900,4802993407,4865000,480294004,4821,48120,48019,4838730,22,18,2,1,0,1,0.004063,113095.333984,1422.540068,"POLYGON ((-98.47326 29.42442, -98.47326 29.423...",95783,95783,E. COMMERCE & S. HACKBERRY,29.42051,-98.473362,0,POINT (-98.47336 29.42051)
480291919001052,15234,1398537,1,480291919001,48029191900,4802993407,4865000,480294004,4821,48120,48019,4838730,18,12,3,2,0,1,0.004010,111617.613281,1416.902264,"POLYGON ((-98.47325 29.42507, -98.47326 29.424...",95783,95783,E. COMMERCE & S. HACKBERRY,29.42051,-98.473362,0,POINT (-98.47336 29.42051)
480291919001051,15235,1398555,0,480291919001,48029191900,4802993407,4865000,480294004,4821,48120,48019,4838730,0,0,0,0,0,0,0.003295,91708.189453,1334.014457,"POLYGON ((-98.47326 29.42560, -98.47325 29.425...",95783,95783,E. COMMERCE & S. HACKBERRY,29.42051,-98.473362,0,POINT (-98.47336 29.42051)
480291919003046,15236,1398572,4,480291919003,48029191900,4802993407,4865000,480294003,4823,48120,48019,4838730,0,0,0,0,0,0,0.004446,123754.332031,1486.665856,"POLYGON ((-98.47167 29.42181, -98.47326 29.421...",95783,95783,E. COMMERCE & S. HACKBERRY,29.42051,-98.473362,0,POINT (-98.47336 29.42051)


In [61]:
gdf_stops_202004_blocks.OBJECTID.value_counts().count()

18684

In [62]:
gdf_stops_202004_blocks.stop_code.value_counts().count()

6491